# Demo

In [ ]:
import os
import time
import sklearn
import joblib
import sounddevice as sd
import numpy as np
import librosa
import pandas as pd
import matplotlib.pyplot as plt
import cv2 as cv

import keras
from keras.preprocessing.image import img_to_array
from tensorflow.keras.applications.vgg16 import preprocess_input
from tensorflow.keras.preprocessing import image as kimage
from tensorflow import keras
from keras_vggface.vggface import VGGFace
from keras_vggface.utils import preprocess_input

from sklearn.neighbors import KDTree

## Audio

In [ ]:
names = ["Alberto", "Alice", "Davide"]

In [ ]:
audiorecog_model = keras.models.load_model('mod_voce.h5')

In [ ]:
def mfcc(input, rate=44100, sampling=1):
    signal = input[::sampling]
    mfcc = librosa.feature.mfcc(y=signal*1.0, sr=int(rate/sampling))
    return mfcc

In [ ]:
def audio_prediction(duration, rec_rate=44100):
    if duration%2!=0:
        duration +=1
    for i in range(int(duration/2)):
        track = sd.rec(int(2 * rec_rate), samplerate=rec_rate, channels=1, blocking=True)
        prediction = np.round(audiorecog_model.predict(mfcc(track.reshape(-1)).reshape(-1,20,173,1)), 3)
        print('Registrazione {}: {}'.format(i+1, names[np.argmax(prediction)]))
        print('---------------------------------')
        print(pd.DataFrame({'Nome':names, 'Probabilità':prediction[0]}))
        print('---------------------------------')
        print('')

In [ ]:
audio_prediction(30)

## Image

In [ ]:
imagerecog_mod=keras.models.load_model('mod_img.h5')

In [ ]:
def image_acquisition(num_images, model):
    face_detector = cv.CascadeClassifier(cv.data.haarcascades+'haarcascade_frontalface_default.xml')
    i = 0
    for i in range(num_images):
        cap = cv.VideoCapture(0)
        time.sleep(2)
        result, img = cap.read()
        faces = face_detector.detectMultiScale(img, minNeighbors=10, minSize=(50,50))
        for (x,y,w,h) in faces:
            face = img[y:y+h,x:x+h,:]
            img_pixels = cv.resize(face, (224, 224)) 
            img_pixels = img_to_array(img_pixels)
            img_pixels = np.expand_dims(img_pixels, axis = 0)
            img_pixels = preprocess_input(img_pixels)
            pred=model.predict(img_pixels)
            print("Foto",i+1,':', names[np.argmax(pred)])
            print('---------------------------------')
            plt.subplot(1,2,1); plt.imshow(img[:,:,-1::-1]); plt.title("Immagine completa:")
            plt.subplot(1,2,2); plt.imshow(face[:,:,-1::-1]); plt.title("Volto:"); plt.show()
            print(pd.DataFrame({'Nome': names, 'Probabilità': np.round(pred, 4)[0]}))
            print('---------------------------------')
            print('')
            i=i+1
    cap.release()

In [ ]:
image_acquisition(4, imagerecog_mod)

## Retrieval

In [ ]:
face_detector = cv.CascadeClassifier(cv.data.haarcascades +'haarcascade_frontalface_default.xml')

def create_new_path(start, paths):
    new_paths = []
    for i in paths:
        end = i.split('content')[1]
        end = end.replace('/', '\\')
        path = start + end
        new_paths.append(path)
    return new_paths

def neural_features_(img):
    global roi_color 
    x = np.asarray(img)
    detector = cv.CascadeClassifier(cv.data.haarcascades +'haarcascade_frontalface_default.xml')
    results = detector.detectMultiScale(x)
    for (x, y, w, h) in results:
        cv.rectangle(img, (x, y), (x + w, y + h), (0, 255, 0), 2)
        roi_color = img[y:y + h, x:x + w]
    x = cv.resize(roi_color, (224,224))
    x = x[:,:,-1::-1].astype('float64')
    x = preprocess_input(x, version = 2) 
    x = np.expand_dims(x, axis=0)
    f = resnet.predict(x)
    return f.flatten()

Caricamento del modello e delle feature estratte dalle immagini del database.  
Creazione dell'albero e modifica ai path per raggiungere le immagini nella nuova directory.

In [ ]:
resnet = VGGFace(model='resnet50', include_top=False, input_shape=(224, 224, 3), pooling='avg')

paths, features = joblib.load('modello_definitivo.joblib')

tree = KDTree(features) 

start = 'data\\thumbnails_features_deduped_publish'
new_paths = create_new_path(start=start, paths = paths)

Acquisizione fotografia con ritaglio del volto.

In [ ]:
cap = cv.VideoCapture(0)
result, img = cap.read()
cap.release()

faces = face_detector.detectMultiScale(img)
for (x,y,w,h) in faces:
    face = img[y:y+h,x:x+h,:]
    
plt.imshow(face[:,:,-1::-1]), plt.show()

Estrazione feature dal volto e ricerca della query nel db.

In [ ]:
query_features = neural_features_(face)
query_features = np.expand_dims(query_features, axis=0)

In [ ]:
start = time.time()
dist, ind = tree.query(query_features, k=10, dualtree=True)
print(time.time()-start)

In [ ]:
fig = plt.figure(figsize=(20,11))
for i in range(10):
    sub = fig.add_subplot(2,5,i+1)
    sub.title.set_text(new_paths[ind[0][i]].split('\\')[3])
    sub.title.set_size(20)
    sub.text(10, -20, ("distance:" + str(round(dist[0][i],3))), fontsize=15)
    fig.suptitle('10 volti più simili con VGGFace-resnet',size=25)
    sub.imshow(kimage.load_img(new_paths[ind[0][i]]), interpolation='bilinear')